05-2 교차 검증과 그리드 서치

In [34]:
#데이터를 불러옵니다
import pandas as pd 

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [35]:
#class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장 

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [36]:
#훈련 세트의 입력 데이터와 타깃 데이터를 train_input, train_target 배열에 저장 
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [37]:
#val 설정해주기 

sub_input, val_input, sub_target, val_target= train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [38]:
# 훈련 세트와 검증 세트의 크기를 확인 

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


원래 있던 훈련세트 5197개가 4157개로 줄고 
검증세트는 1040개가 되었음 

In [39]:
#모델을 만들고 평가 

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


이 모델은 훈련세트에 과대적합되었기에 매개변수를 바꾸어서 더 좋은 모델을 찾아야한다! 

In [40]:
#교차검증 

from sklearn.model_selection import cross_validate
scores= cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0066328 , 0.00592399, 0.00619078, 0.00614119, 0.00572181]), 'score_time': array([0.00062275, 0.00057697, 0.00050664, 0.00050163, 0.00053978]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate()는 기본적으로 5-폴드 교차 검증을 수행함 
cv 매개변수에서 폴드 수를 바꿀 수도 있음

In [41]:
import numpy as np
#검증 폴드의 점수
print(np.mean(scores['test_score']))

0.855300214703487


In [42]:
# 회귀모델이면 KFold 
#근데 분류모델이기에 StratifiedKFold

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [43]:
#훈련 세트를 섞은 후 10-폴드 교차검증을 수행 

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores= cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [44]:
from sklearn.model_selection import GridSearchCV
params= {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [45]:
#GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달, 그리드객체 생성 

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

#n_jobs은 병렬실행에 사용할 CPU 코어수를 지정 
#-1, 시스템에 있는 모든 코어를 사용한다는 뜻

In [46]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [47]:
dt= gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [48]:
#베스트 params의 값 

print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [49]:
#5번의 교차검증으로 얻은 점수 출력 

print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [50]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수를 지정 
2. 훈련세트에서 그리드 서치를 수행 -> 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾고 그리드 서치 객체에 저장 
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련-> 그리드 서치 객체에 저장

In [51]:
#조금 더 복잡하게 

# 0.0001에서 0.001에 도달할때 까지 0.0001씩 더한 배열
#두번째 매개변수 미포함으로 배열의 원소 9개 
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),          
#5에서 20까지 1씩 증가하면서 15개의 값을 만듦 
          'max_depth':range(5, 20 ,1),
#2부터 100까지 10씩 증가하면서 10개의 값을 만듦
          'min_samples_split': range(2,100,10)
          }

이 매개변수로 수행할 교차 검증의 횟수는 9 x 15 x 10 = 1350개

5-폴드 교차 검증 수행 하므로 만들어지는 모델 수 6750개 

In [52]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [53]:
#최상의 매개변수 조합확인 

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [54]:
#교차점수의 점수 
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [55]:
##랜덤서치 


from scipy.stats import uniform, randint
#uniform, randint은 주어진 범위에서 고르게 값을 뽑음 
#균등분포에서 샘플링한다! 
#uniform - 정수값 , randint- 실수값 

In [56]:
rgen = randint(0,10)
rgen.rvs(10)

array([2, 5, 5, 0, 0, 4, 1, 9, 9, 8])

In [57]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107,  98, 100,  93, 107,  94,  97, 102, 104,  98]))

In [58]:
ugen= uniform(0,1)
ugen.rvs(10)

array([0.62501854, 0.42956163, 0.81636074, 0.96931767, 0.62278377,
       0.99422061, 0.7237511 , 0.37488438, 0.33841623, 0.70352065])

In [59]:
params = {
    #0.0001에서 0.001사이의 실숫값을 샘플링 
    'min_impurity_decrease': uniform(0.0001, 0.001),
    #20에서 50사이의 정수값을 샘플링 
          'max_depth': randint(20,50),
    #2에서 25사리의 정수값을 샘플링 
          'min_samples_split': randint(2,25),
    #1에서 25사이의 정수를 샘플링
          'min_samples_leaf': randint(1,25)
          }

#샘플링 횟수는 RandomizedSearchCV의 n_iter 매개변수에 지정한다 

In [60]:
from sklearn.model_selection import RandomizedSearchCV

#params에서 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 
#교차검증을 수행하고 최적의 매개변수 조합을 찾음 
gs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                       params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)



RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa47e050a90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa47e050e50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa47fe4ab10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa47e050710>},
                   random_state=42)

In [61]:
#최적의 매개변수 조합을 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [62]:
#최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [63]:
#이 모델 최종모델로 선정 후 테스트 성능확인 

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


확인문제 03 

In [64]:
gs= RandomizedSearchCV(DecisionTreeClassifier(splitter= 'random',
                       random_state=42), params, n_iter=100,
                       n_jobs=-1, random_state= 42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt= gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
